In [1]:
from preprocessing import *
from sklearn.model_selection import train_test_split, KFold, ParameterGrid
import argparse
from model import *
from train import test
import torch.optim as optim

from MatrixVectorizer import *


In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = np.loadtxt(lr_data_path, delimiter=',')
hr_train_data = np.loadtxt(hr_data_path, delimiter=',')
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 260) for row in hr_train_data])
num_samples = hr_train_data_vectorized.shape[0]
split = int(num_samples * 0.8)

In [3]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [4]:
num_splt = 3
epochs = 15
lr = 0.0001
lmbda = 500
lr_dim = 160
hr_dim = 320
hidden_dim = 1024
padding = 30
dropout = 0.15
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


lr_train_data_vectorized.shape=(168, 160, 160)
lr_train.shape=(134, 160, 160)
lr_test.shape=(34, 160, 160)
hr_train_data_vectorized.shape=(168, 260, 260)
hr_train.shape=(134, 260, 260)
hr_test.shape=(34, 260, 260)


In [5]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [6]:
# ks = [0]
ks = [0.7, 0.3]
model = GSRNet(ks, args)

In [7]:
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args, subjects_adj_test, subjects_ground_truth_test):
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []

    model.train()
    for lr,hr in zip(subjects_adj,subjects_labels):      
      lr = torch.from_numpy(lr).type(torch.FloatTensor)
      hr = torch.from_numpy(hr).type(torch.FloatTensor)
      
      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      # model_outputs  = unpad(model_outputs, args.padding)
      # weights = unpad(model.layer.weights, args.padding)
      
      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      # print the shapes of the outputs
      # print(f"{net_outs.shape} ; {start_gcn_outs.shape}")
      # print(f"{model.layer.weights.shape} ; {U_hr.shape}")
      # print(f"{model_outputs.shape} ; {hr.shape}")
      
      # loss = criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + args.lmbda * criterion(model_outputs, hr) 
      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, padded_hr) 

      
      error = criterion(model_outputs, padded_hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      
  
    model.eval()
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error))
    test(model, subjects_adj_test, subjects_ground_truth_test, args)
    all_epochs_loss.append(np.mean(epoch_loss))

In [8]:
# Define sets of hyperparameters for tuning
learning_rates = [0.0001, 0.0002]
lambdas = [1000, 5000, 10000]
hidden_dims = [256, 512, 1024]

num_splt = 3
epochs = 10
# lr = 0.00005
# lmbda = 5000
lr_dim = 160
hr_dim = 320
# hidden_dim = 512
padding = 30
ks = [0.7, 0.3]

hyperparam_grid = {
    'lr': learning_rates,
    'lmbda': lambdas,
    'hidden_dim': hidden_dims
}

best_hyperparams = None
best_val_loss = float('inf')

for hyperparams in ParameterGrid(hyperparam_grid):
    print(f"Training with hyperparams: {hyperparams}")

    train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
    print('Held out test score:')
    test(model, held_out_subjects_adj, held_out_subjects_labels, args)
    print('------------------------------')

Epoch:  1 Loss:  3.6596158515201527 Error:  0.15330434565463763
Test error MAE:  372.8449371493525
Epoch:  2 Loss:  1.9087520810995209 Error:  0.14469659378689326
Test error MAE:  372.84525650342306
Epoch:  3 Loss:  1.5236091921838482 Error:  0.14320210626955782
Test error MAE:  372.853780942493
Epoch:  4 Loss:  1.2878693797615137 Error:  0.1424808210871193
Test error MAE:  372.8468992928664
Epoch:  5 Loss:  1.1263392005073891 Error:  0.1414220974351583
Test error MAE:  372.8462562004725
Epoch:  6 Loss:  1.0410796604799422 Error:  0.14068403078264066
Test error MAE:  372.8432337535752
Epoch:  7 Loss:  0.9679718091246787 Error:  0.13984059575903282
Test error MAE:  372.83922985692817
Epoch:  8 Loss:  0.917169061269653 Error:  0.1387727681672975
Test error MAE:  372.8376896282037
Epoch:  9 Loss:  0.8739230063524139 Error:  0.1381598791379607
Test error MAE:  372.83738103674517
Epoch:  10 Loss:  0.8450608414210631 Error:  0.13723236732603458
Test error MAE:  372.83630076845486
Epoch:  11 